# Advanced usage

In [ ]:
# Useful for debugging
# %load_ext autoreload
# %autoreload 2

# Nicer plots
%config InlineBackend.figure_format = 'retina'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from pytao import Tao, TaoModel, util, run_tao
from pytao.tao_ctypes.evaluate import evaluate_tao
import os

## Inititialize Tao

In [ ]:
INPUT_FILE = os.path.expandvars(
    "$ACC_ROOT_DIR/bmad-doc/tao_examples/csr_beam_tracking/tao.init"
)
assert os.path.exists(os.path.expandvars(INPUT_FILE))

## run_tao

In [ ]:
M = run_tao(input_file=INPUT_FILE, ploton=False)
M

## evaluate_tao

In [ ]:
res = evaluate_tao(
    settings={"space_charge_com:ds_track_step": 0.001},
    input_file=INPUT_FILE,
    run_commands=["set global track_type=beam"],
    expressions=["lat::orbit.x[FF.PIP02A]", "beam::norm_emit.x[end]"],
    ploton=False,
    archive_csr_wake=True,
    beam_archive_path=".",
)
res

In [ ]:
from pmd_beamphysics import ParticleGroup, particle_paths
from h5py import File

afile = res["beam_archive"]
h5 = File(afile, "r")
ppaths = particle_paths(h5)
ppaths

Note that csr_wake is in here

In [ ]:
list(h5)

In [ ]:
P = ParticleGroup(h5[ppaths[-1]])
P.plot("delta_t", "delta_pz", bins=200)

In [ ]:
os.remove(afile)

## evaluate examples

In [ ]:
M.evaluate("lat::orbit.x[end]")

## csr_wake diagnostic file

In [ ]:
from pytao.misc.csr import read_csr_wake_data_h5, process_csr_wake_data
import numpy as np

csr_wake is stored in the HDF5 file

In [ ]:
cdat = read_csr_wake_data_h5(h5)

This is the actual data

In [ ]:
dat = cdat["3:FF.BEN01"]["data"]
dat.shape

This will process into a more useful form for data analysis.

In [ ]:
pdat = process_csr_wake_data(cdat)

pdat["s_position"]

In [ ]:
from pytao.misc.csr_plot import plot_csr_wake, plot_csr_stats
from pytao.misc.csr import csr_wake_stats_at_step

In [ ]:
from ipywidgets import interact


def plot1(step=0):
    plot_csr_wake(pdat, step=step)


nstep = len(pdat["z"])
interact(plot1, step=(0, nstep - 1, 1));

Check the s-position for each step

In [ ]:
plt.plot(pdat["s_position"], marker=".");

Another convenience plot

In [ ]:
plot_csr_stats(pdat)

Cleanup

In [ ]:
os.remove("wake.dat")